In [74]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from fbprophet import Prophet
import pycountry
import plotly.express as px
from collections import namedtuple

In [ ]:
# Improting India and US data

In [75]:
url1 = 'https://api.covid19india.org/csv/latest/state_wise_daily.csv'
st_ws_daily = pd.read_csv(url1)

st_ws_daily.to_csv('st_ws_daily')
df = pd.melt(st_ws_daily,id_vars=['Date','Status'],var_name='Province/State',value_name='Cases')
df['Date']=pd.to_datetime(df['Date'])
df1=df.loc[df['Province/State']!='TT']
df1.head()

,Date,Status,Province/State,Cases
405,2020-03-14,Confirmed,AN,0
406,2020-03-14,Recovered,AN,0
407,2020-03-14,Deceased,AN,0
408,2020-03-15,Confirmed,AN,0
409,2020-03-15,Recovered,AN,0


In [64]:
st_ws_daily.head()

,Date,Status,TT,AN,AP,AR,AS,BR,CH,CT,...,PB,RJ,SK,TN,TG,TR,UP,UT,WB,UN
0,14-Mar-20,Confirmed,81,0,1,0,0,0,0,0,...,1,3,0,1,1,0,12,0,0,0
1,14-Mar-20,Recovered,9,0,0,0,0,0,0,0,...,0,1,0,0,0,0,4,0,0,0
2,14-Mar-20,Deceased,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15-Mar-20,Confirmed,27,0,0,0,0,0,0,0,...,0,1,0,0,2,0,1,0,0,0
4,15-Mar-20,Recovered,4,0,0,0,0,0,0,0,...,0,2,0,0,1,0,0,0,0,0


In [114]:
states=df1['Province/State'].unique()
states

array(['AN', 'AP', 'AR', 'AS', 'BR', 'CH', 'CT', 'DN', 'DD', 'DL', 'GA',
       'GJ', 'HR', 'HP', 'JK', 'JH', 'KA', 'KL', 'LA', 'LD', 'MP', 'MH',
       'MN', 'ML', 'MZ', 'NL', 'OR', 'PY', 'PB', 'RJ', 'SK', 'TN', 'TG',
       'TR', 'UP', 'UT', 'WB', 'UN'], dtype=object)

In [65]:
st_ws_daily.shape

(405, 41)

In [66]:
df1.isnull().sum()

Date              0
Status            0
Province/State    0
Cases             0
dtype: int64

In [67]:
df1.nunique()

Date               135
Status               3
Province/State      38
Cases             1318
dtype: int64

In [76]:
print(df1['Date'].min(),df1['Date'].max())

2020-03-14 00:00:00 2020-07-26 00:00:00


In [77]:
df1['SNo']='1'
df1['Country']='India'

Confirmed_df=df1.loc[df1['Status']=='Confirmed']
Recovered_df=df1.loc[df1['Status']=='Recovered']
Deceased_df=df1.loc[df1['Status']=='Deceased']
all_countries = df1['Country'].unique()

Confirmed_df = Confirmed_df.drop(columns = ['Status'])
Recovered_df = Recovered_df.drop(columns = ['Status'])
Deceased_df = Deceased_df.drop(columns = ['Status'])

confirmed_df = Confirmed_df.rename(columns={'Cases': 'Confirmed'})
deaths_df = Deceased_df.rename(columns={'Cases': 'Deaths'})
recovered_df = Recovered_df.rename(columns={'Cases': 'Recovered'})

confirmed_df = confirmed_df[['SNo', 'Date','Province/State', 'Country', 'Confirmed']]
deaths_df = deaths_df[['SNo', 'Date','Province/State', 'Country', 'Deaths']]
recovered_df = recovered_df[['SNo', 'Date','Province/State', 'Country', 'Recovered']]

/Users/deepakpareek/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/deepakpareek/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [70]:
print(confirmed_df.shape)
print(deaths_df.shape)
print(recovered_df.shape)

(5130, 5)
(5130, 5)
(5130, 5)


In [78]:
days_to_forecast = 7 # changable
first_forecasted_date = sorted(list(set(df1['Date'].values)))[-days_to_forecast]

print('The first date to perform forecasts for is: ' + str(first_forecasted_date))

The first date to perform forecasts for is: 2020-07-20T00:00:00.000000000


In [ ]:
# Covid Cases Prediction using Prophet 

In [79]:
def make_forecasts(all_countries, confirmed_df, deaths_df, recovered_df, days_to_forecast, first_forecasted_date, mode):
    forecast_dfs = []
    absolute_errors = [] # collate absolute errors so that we can find MAE later on
    counter = 0 # arbitrary counter to output snippet of result_val_df only once, when it is 1
    for country in all_countries:
        try:
            assert(country in confirmed_df['Country'].values)
            print('Country ' + str(country) + ' is listed! ')
            country_confirmed_df = confirmed_df[(confirmed_df['Country'] == country)]
            country_deaths_df = deaths_df[(deaths_df['Country'] == country)]
            country_recovered_df = recovered_df[(recovered_df['Country'] == country)]
            country_dfs = [('Confirmed', country_confirmed_df), 
                           ('Deaths', country_deaths_df), 
                           ('Recovered', country_recovered_df)]
            states_in_country = country_confirmed_df['Province/State'].unique()
            for state in states_in_country:
                try:
                    state_dfs = [] # to store forecasts for Confirmed, Deaths and Recovered

                    assert(state in country_confirmed_df['Province/State'].values)

                    # make forecasts for each case type (Confirmed, Deaths, Recovered)
                    for country_df_tup in country_dfs:
                        case_type = country_df_tup[0]
                        country_df = country_df_tup[1]
                        state_df = country_df[(country_df['Province/State'] == state)]

                        # data preparation for forecast with Prophet at state level
                        state_df = state_df[['Date', case_type]]
                        state_df.columns = ['ds','y']
                        state_df['ds'] = pd.to_datetime(state_df['ds'])

                        state_df_val = state_df[(state_df['ds'] >= pd.to_datetime(first_forecasted_date))] # validation set
                        state_df = state_df[(state_df['ds'] < pd.to_datetime(first_forecasted_date))] # train set

                        if mode == 'default':
                            m = Prophet()
                        elif mode == 'custom':
                            m = Prophet(daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=False)
                            m.add_seasonality(name='monthly', period=30.5, fourier_order=10)
                            m.add_seasonality(name='weekly', period=7, fourier_order=21)
                            m.add_seasonality(name='daily', period=1, fourier_order=3)
                        m.fit(state_df)
                        future = m.make_future_dataframe(periods=days_to_forecast)
                        forecast = m.predict(future)

                        # evaluate forecasts with validation set and save absolute errors to absolute_errors
                        forecast_df = forecast[['ds', 'yhat']]
                        result_df = forecast_df[(forecast_df['ds'] >= pd.to_datetime(first_forecasted_date))]
                        result_val_df = result_df.merge(state_df_val, on=['ds'])
                        result_val_df['abs_diff'] = (result_val_df['y'] - result_val_df['yhat']).abs()
                        absolute_errors += list(result_val_df['abs_diff'].values)

                        # the following allows the user to check the output at particular checkpoints. Feel free to change!
                        if counter == 1:
                            print('Printing snippet of result_val_df: \n')
                            print(result_val_df)
                            print(absolute_errors)
                        counter += 1

                        # save results to dataframe
                        forecast_df['Province/State'] = state
                        forecast_df['Country/Region'] = country
                        forecast_df.rename(columns={'yhat':case_type}, inplace=True)
                        state_dfs += [forecast_df.tail(days_to_forecast)]

                    merged_df = state_dfs[0].merge(state_dfs[1],on=['ds', 'Province/State', 'Country/Region']).merge(state_dfs[2],on=['ds', 'Province/State', 'Country/Region'])
                    forecast_dfs += [merged_df]
                except:
                    continue
        except:
            print('Country ' + str(country) + ' is not listed! ')
            continue
    results_tup = namedtuple('results_tup', ['forecast_dfs', 'absolute_errors'])
    return results_tup(forecast_dfs, absolute_errors)

In [80]:
forecast_results = make_forecasts(all_countries, confirmed_df, deaths_df, recovered_df, days_to_forecast, first_forecasted_date, 'default')
absolute_errors = forecast_results.absolute_errors

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Country India is listed! 


/Users/deepakpareek/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/deepakpareek/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonal

Printing snippet of result_val_df: 

          ds  yhat  y  abs_diff
0 2020-07-20   0.0  0       0.0
1 2020-07-21   0.0  0       0.0
2 2020-07-22   0.0  0       0.0
3 2020-07-23   0.0  0       0.0
4 2020-07-24   0.0  0       0.0
5 2020-07-25   0.0  0       0.0
6 2020-07-26   0.0  0       0.0
[0.6957141899952752, 0.5102255706448862, 4.826952009978315, 14.148121235423854, 13.856782066662081, 26.872144929525092, 22.44155329368364, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

In [84]:
forecasts_final = pd.concat(forecast_results.forecast_dfs, axis=0)
forecasts_final.sort_values(by='ds')
forecasts_final = forecasts_final[['ds', 'Province/State', 'Country/Region', 'Deaths', 'Recovered']]
forecasts_final.rename(columns={'ds':'ObservationDate'}, inplace=True)
for case_type in ['Deaths', 'Recovered']:
    forecasts_final[case_type] = forecasts_final[case_type].round() # round forecasts to integer as humans cannot be floats
    #forecasts_final[forecasts_final[case_type] < 0] = 0 # replace negative forecasts to zero

forecasts_final

,ObservationDate,Province/State,Country/Region,Deaths,Recovered
0,2020-07-20,AN,India,0.0,3.0
1,2020-07-21,AN,India,0.0,4.0
2,2020-07-22,AN,India,0.0,3.0
3,2020-07-23,AN,India,0.0,4.0
4,2020-07-24,AN,India,0.0,3.0
...,...,...,...,...,...
2,2020-07-22,UN,India,0.0,0.0
3,2020-07-23,UN,India,0.0,0.0
4,2020-07-24,UN,India,0.0,0.0
5,2020-07-25,UN,India,0.0,0.0


In [85]:
forecasts_final['PREDICTION'] = forecasts_final['Deaths']+forecasts_final['Recovered']
forecasts_final = forecasts_final.rename(columns={'ObservationDate':'DATE','Province/State': 'STATE_CD'})
forecasts_final = forecasts_final [['DATE','STATE_CD','PREDICTION']]
forecasts_final

,DATE,STATE_CD,PREDICTION
0,2020-07-20,AN,3.0
1,2020-07-21,AN,4.0
2,2020-07-22,AN,3.0
3,2020-07-23,AN,4.0
4,2020-07-24,AN,3.0
...,...,...,...
2,2020-07-22,UN,0.0
3,2020-07-23,UN,0.0
4,2020-07-24,UN,0.0
5,2020-07-25,UN,0.0


In [87]:
DL=forecasts_final.loc[forecasts_final['STATE_CD']=='DL']
DL

,DATE,STATE_CD,PREDICTION
0,2020-07-20,DL,2652.0
1,2020-07-21,DL,2674.0
2,2020-07-22,DL,2777.0
3,2020-07-23,DL,3079.0
4,2020-07-24,DL,2834.0
5,2020-07-25,DL,3234.0
6,2020-07-26,DL,2930.0


In [89]:
MH=forecasts_final.loc[forecasts_final['STATE_CD']=='MH']
MH

,DATE,STATE_CD,PREDICTION
0,2020-07-20,MH,4309.0
1,2020-07-21,MH,4250.0
2,2020-07-22,MH,4287.0
3,2020-07-23,MH,4712.0
4,2020-07-24,MH,4721.0
5,2020-07-25,MH,4592.0
6,2020-07-26,MH,4415.0


In [91]:
KA=forecasts_final.loc[forecasts_final['STATE_CD']=='KA']
KA

,DATE,STATE_CD,PREDICTION
0,2020-07-20,KA,1013.0
1,2020-07-21,KA,1052.0
2,2020-07-22,KA,1072.0
3,2020-07-23,KA,1119.0
4,2020-07-24,KA,1145.0
5,2020-07-25,KA,1159.0
6,2020-07-26,KA,1191.0


In [94]:
TN=forecasts_final.loc[forecasts_final['STATE_CD']=='TN']
TN

,DATE,STATE_CD,PREDICTION
0,2020-07-20,TN,4212.0
1,2020-07-21,TN,4497.0
2,2020-07-22,TN,4575.0
3,2020-07-23,TN,4697.0
4,2020-07-24,TN,4678.0
5,2020-07-25,TN,4725.0
6,2020-07-26,TN,4776.0


In [102]:
RJ=forecasts_final.loc[forecasts_final['STATE_CD']=='RJ']
RJ

,DATE,STATE_CD,PREDICTION
0,2020-07-20,RJ,421.0
1,2020-07-21,RJ,410.0
2,2020-07-22,RJ,439.0
3,2020-07-23,RJ,419.0
4,2020-07-24,RJ,459.0
5,2020-07-25,RJ,451.0
6,2020-07-26,RJ,428.0


In [88]:
df1.head()

,Date,Status,Province/State,Cases,SNo,Country
405,2020-03-14,Confirmed,AN,0,1,India
406,2020-03-14,Recovered,AN,0,1,India
407,2020-03-14,Deceased,AN,0,1,India
408,2020-03-15,Confirmed,AN,0,1,India
409,2020-03-15,Recovered,AN,0,1,India


In [ ]:
# For US

In [95]:
us=pd.read_csv('covid_confirmed_usafacts.csv')
us_df = pd.melt(us,id_vars=['countyFIPS','County Name','State','stateFIPS'],var_name='Date',value_name='Cases')
us_df['Date']=pd.to_datetime(us_df['Date'])
us_df1=us_df.groupby(['Date','State']).Cases.sum().reset_index()
us_df1['SNo']='1'
us_df1['Country']='US'
confirmed_df = us_df1.rename(columns={'Cases': 'Confirmed','State':'Province/State'})

In [96]:
us_df.nunique()

countyFIPS      3146
County Name     1882
State             51
stateFIPS         51
Date             184
Cases          12102
dtype: int64

In [97]:
print(us_df['Date'].min(),us_df['Date'].max())

2020-01-22 00:00:00 2020-07-23 00:00:00


In [98]:
us_df.isnull().sum()

countyFIPS     184
County Name    184
State          184
stateFIPS      184
Date             0
Cases          183
dtype: int64

In [99]:
confirmed_df2=confirmed_df.loc[confirmed_df['Date']<'2020-07-20']
all_countries = confirmed_df2['Country'].unique()

In [ ]:
confirmed_df2.head()

In [100]:
def make_forecasts(all_countries, confirmed_df,  days_to_forecast, first_forecasted_date, mode):
    forecast_dfs = []
    absolute_errors = [] # collate absolute errors so that we can find MAE later on
    counter = 0 # arbitrary counter to output snippet of result_val_df only once, when it is 1
    for country in all_countries:
        try:
            assert(country in confirmed_df['Country'].values)
            print('Country ' + str(country) + ' is listed! ')
            country_confirmed_df = confirmed_df[(confirmed_df['Country'] == country)]
            #country_deaths_df = deaths_df[(deaths_df['Country'] == country)]
            #country_recovered_df = recovered_df[(recovered_df['Country'] == country)]
            country_dfs = [('Confirmed', country_confirmed_df)]
            states_in_country = country_confirmed_df['Province/State'].unique()
            for state in states_in_country:
                try:
                    state_dfs = [] # to store forecasts for Confirmed, Deaths and Recovered

                    assert(state in country_confirmed_df['Province/State'].values)

                    # make forecasts for each case type (Confirmed, Deaths, Recovered)
                    for country_df_tup in country_dfs:
                        case_type = country_df_tup[0]
                        country_df = country_df_tup[1]
                        state_df = country_df[(country_df['Province/State'] == state)]

                        # data preparation for forecast with Prophet at state level
                        state_df = state_df[['Date', case_type]]
                        state_df.columns = ['ds','y']
                        state_df['ds'] = pd.to_datetime(state_df['ds'])

                        state_df_val = state_df[(state_df['ds'] >= pd.to_datetime(first_forecasted_date))] # validation set
                        state_df = state_df[(state_df['ds'] < pd.to_datetime(first_forecasted_date))] # train set

                        if mode == 'default':
                            m = Prophet()
                        elif mode == 'custom':
                            m = Prophet(daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=False)
                            m.add_seasonality(name='monthly', period=30.5, fourier_order=10)
                            m.add_seasonality(name='weekly', period=7, fourier_order=21)
                            m.add_seasonality(name='daily', period=1, fourier_order=3)
                        m.fit(state_df)
                        future = m.make_future_dataframe(periods=days_to_forecast)
                        forecast = m.predict(future)

                        # evaluate forecasts with validation set and save absolute errors to absolute_errors
                        forecast_df = forecast[['ds', 'yhat']]
                        result_df = forecast_df[(forecast_df['ds'] >= pd.to_datetime(first_forecasted_date))]
                        result_val_df = result_df.merge(state_df_val, on=['ds'])
                        result_val_df['abs_diff'] = (result_val_df['y'] - result_val_df['yhat']).abs()
                        absolute_errors += list(result_val_df['abs_diff'].values)

                        # the following allows the user to check the output at particular checkpoints. Feel free to change!
                        if counter == 1:
                            print('Printing snippet of result_val_df: \n')
                            print(result_val_df)
                            print(absolute_errors)
                        counter += 1

                        # save results to dataframe
                        forecast_df['Province/State'] = state
                        forecast_df['Country/Region'] = country
                        forecast_df.rename(columns={'yhat':case_type}, inplace=True)
                        state_dfs += [forecast_df.tail(days_to_forecast)]

                    #merged_df = state_dfs[0].merge(state_dfs[1],on=['ds', 'Province/State', 'Country/Region']).merge(state_dfs[2],on=['ds', 'Province/State', 'Country/Region'])
                    merged_df = state_dfs[0]
                    forecast_dfs += [merged_df]
                except:
                    continue
        except:
            print('Country ' + str(country) + ' is not listed! ')
            continue
    results_tup = namedtuple('results_tup', ['forecast_dfs', 'absolute_errors'])
    return results_tup(forecast_dfs, absolute_errors)

In [101]:
forecast_results_us = make_forecasts(all_countries, confirmed_df2, days_to_forecast, first_forecasted_date, 'default')
absolute_errors = forecast_results_us.absolute_errors

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Country US is listed! 


/Users/deepakpareek/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/deepakpareek/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonal

Printing snippet of result_val_df: 

Empty DataFrame
Columns: [ds, yhat, y, abs_diff]
Index: []
[]


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

In [103]:
forecasts_final_us = pd.concat(forecast_results_us.forecast_dfs, axis=0)
forecasts_final_us.sort_values(by='ds')
forecasts_final_us = forecasts_final_us[['ds', 'Province/State', 'Country/Region', 'Confirmed']]
forecasts_final_us.rename(columns={'ds':'ObservationDate'}, inplace=True)
for case_type in ['Confirmed']:
    forecasts_final_us[case_type] = forecasts_final_us[case_type].round() # round forecasts to integer as humans cannot be floats
    #forecasts_final_us[forecasts_final[case_type] < 0] = 0 # replace negative forecasts to zero

forecasts_final_us

,ObservationDate,Province/State,Country/Region,Confirmed
180,2020-07-20,AK,US,1645.0
181,2020-07-21,AK,US,1674.0
182,2020-07-22,AK,US,1704.0
183,2020-07-23,AK,US,1733.0
184,2020-07-24,AK,US,1766.0
...,...,...,...,...
182,2020-07-22,WY,US,2164.0
183,2020-07-23,WY,US,2195.0
184,2020-07-24,WY,US,2226.0
185,2020-07-25,WY,US,2255.0


In [104]:
forecasts_final_us = forecasts_final_us.rename(columns={'ObservationDate':'DATE','Province/State': 'STATE_CD','Confirmed':'PREDICTION'})
forecasts_final_us= forecasts_final_us[['DATE','STATE_CD','PREDICTION']]
forecasts_final_us

,DATE,STATE_CD,PREDICTION
180,2020-07-20,AK,1645.0
181,2020-07-21,AK,1674.0
182,2020-07-22,AK,1704.0
183,2020-07-23,AK,1733.0
184,2020-07-24,AK,1766.0
...,...,...,...
182,2020-07-22,WY,2164.0
183,2020-07-23,WY,2195.0
184,2020-07-24,WY,2226.0
185,2020-07-25,WY,2255.0


In [105]:
NY=forecasts_final_us.loc[forecasts_final_us['STATE_CD']=='NY']
NY

,DATE,STATE_CD,PREDICTION
180,2020-07-20,NY,407079.0
181,2020-07-21,NY,407383.0
182,2020-07-22,NY,408275.0
183,2020-07-23,NY,409101.0
184,2020-07-24,NY,409947.0
185,2020-07-25,NY,410989.0
186,2020-07-26,NY,411448.0


In [106]:
NJ=forecasts_final_us.loc[forecasts_final_us['STATE_CD']=='NJ']
NJ

,DATE,STATE_CD,PREDICTION
180,2020-07-20,NJ,177336.0
181,2020-07-21,NJ,177649.0
182,2020-07-22,NJ,177852.0
183,2020-07-23,NJ,178241.0
184,2020-07-24,NJ,178544.0
185,2020-07-25,NJ,178847.0
186,2020-07-26,NJ,179102.0


In [108]:
PA=forecasts_final_us.loc[forecasts_final_us['STATE_CD']=='PA']
PA

,DATE,STATE_CD,PREDICTION
180,2020-07-20,PA,99858.0
181,2020-07-21,PA,100466.0
182,2020-07-22,PA,101071.0
183,2020-07-23,PA,101728.0
184,2020-07-24,PA,102445.0
185,2020-07-25,PA,103134.0
186,2020-07-26,PA,103713.0


In [51]:
forecasts_final_us['DATE'].min(),forecasts_final_us['DATE'].max()

(Timestamp('2020-07-20 00:00:00'), Timestamp('2020-07-26 00:00:00'))

In [52]:
forecasts_final['DATE'].min(),forecasts_final['DATE'].max()

(Timestamp('2020-07-20 00:00:00'), Timestamp('2020-07-26 00:00:00'))